In [1]:
import pandas as pd
import numpy as np
from joblib import dump 

# Preparo los features

In [2]:
installs = pd.read_csv('../datos/installs.csv', dtype ={'device_language' : 'str', 'click_hash': object, 'wifi': object, 'trans_id': object})

In [3]:
# No tiene sentido quedarse con el pais, porque es uno solo
#No tiene sentido quedarse con la columna trans_id. El 98% de los valores son Null
#No tiene sentido quedarse con click_hash, el 99% de los valors son null
#No tiene sentido quedarse con event_uuid, el 78% de los valores son null
#No tiene sentido quedarse con kind, el 78% de los valores son null
#No tiene sentido quedarse con wifi, el 48% de los valores son null
#No tiene sentido quedarse con device_brand. El 42% son null
#No tiene sentido quedarse con la columna user_agent El 31% de los valores son Null
#¿Tiene sentido quedarse con la columna session_user_agent? El 3% de los valores son Null
# session_user_agent tiene el 3% de los valores null. Borro esas filas
del installs['trans_id']
del installs['device_countrycode']
del installs['click_hash']
del installs['event_uuid']
del installs['kind']
del installs['wifi']
del installs['device_brand']
del installs['user_agent']
del installs['session_user_agent']

##Borro las filas que tienen null en device_model y device_language. Juntas son el 5% de las filas en null
installs = installs[pd.notnull(installs['device_model'])]
installs = installs[pd.notnull(installs['device_language'])]

In [4]:
installs['created'] = pd.to_datetime(installs['created'], errors = 'coerce')
installs = installs.sort_values(by = ['ref_hash', 'created'])

In [5]:
events = pd.read_csv('../datos/events.csv',
                     dtype={
                         'device_countrycode': 'category', 
                         'device_city': 'category',
                         'ref_type': 'int64',
                         'application_id': 'int64',
                         'device_os_version': 'category',
                         'device_brand': 'category',
                         'device_model': 'float64',
                         'session_user_agent': 'category',
                         'trans_id': 'category',
                         'user_agent': 'category',
                         'carrier': 'category',
                         'device_os': 'category',
                         'device_os_version': 'category',
                         'device_language': 'category',
                         'connection_type': 'category',
                         'wifi': 'bool'
                     })

In [6]:
del events['attributed']
del events['device_countrycode'] #Borrada porque hay un solo país
del events['trans_id'] #Borrada por tener muy pocos valores (37000/7000000)
del events['event_uuid'] #Borrada por tener todos los valores distintos
del events['device_os_version'] #Borrado por tener 70% de los valores nulos.
del events['device_brand'] #Borrado por tener 67% de los valores nulos
del events['device_city'] #Borrado por tener 76% de los valores nulos
del events['user_agent'] #Borrado por tener 57% de los valores nulos
del events['carrier'] #Borrado por tener 75% de los valores nulos
del events['device_os'] #Borrado por tener 76% de los valores nulos
del events['connection_type'] #Borrado por tener 77% de los valores nulos
del events['session_user_agent']
#------------------------------------------------------------------
events = events[pd.notnull(events['kind'])] #Borrados las filas nulas (0,5%)
#------------------------------------------------------------------
events = events[pd.notnull(events['device_model'])]

In [7]:
events['ref_hash'].value_counts().count()

348229

In [8]:
installs['ref_hash'].value_counts().count()

373799

In [9]:
installs.head()

,created,application_id,ref_type,ref_hash,attributed,implicit,device_model,ip_address,device_language
46742,2019-04-21 19:17:47.657,77,1891515180541284343,40621409780134,False,False,2.019322e+18,990380024993756535,3.3013777759776993e+18
398921,2019-04-18 21:11:50.326,121,1891515180541284343,41863526108385,False,True,1.658417e+18,7530145772806895848,6.977049253562486e+18
249204,2019-04-18 21:11:51.966,121,1891515180541284343,41863526108385,False,False,1.658417e+18,7530145772806895848,6.977049253562486e+18
112218,2019-04-18 21:17:11.946,65,1891515180541284343,41863526108385,False,False,1.658417e+18,7530145772806895848,6.977049253562486e+18
112200,2019-04-18 21:17:16.531,65,1891515180541284343,41863526108385,False,True,1.658417e+18,7530145772806895848,6.977049253562486e+18


In [10]:
eventos = events.groupby('ref_hash').agg({'wifi' : 'mean', 'application_id' : 'count', 'kind' : 'nunique', 'event_id' : 'count'}).reset_index()

In [ ]:
events['mode_model'] = events.groupby('ref_hash')['device_model'].agg(lambda x: x.value_counts().index[0])
events['mode_lang'] = events.groupby('ref_hash')['device_language'].agg(lambda x: x.value_counts().index[0])

In [ ]:
installs['total_apps'] = installs.groupby('ref_hash')['application_id'].transform('count')
installs['%implicit'] = installs.groupby('ref_hash')['implicit'].transform('mean')
installs['%attributed'] = installs.groupby('ref_hash')['attributed'].transform('mean')
installs['most_freq_lang'] = installs.groupby('ref_hash')['device_language'].transform(lambda x: x.mode().iloc[0])
installs['most_freq_app'] = installs.groupby('ref_hash')['application_id'].transform(lambda x: x.mode().iloc[0])
installs['model'] = installs.groupby('ref_hash')['device_model'].transform(lambda x: x.mode().iloc[0])
installs['ip'] = installs.groupby('ref_hash')['ip_address'].transform(lambda x: x.mode().iloc[0])

In [ ]:
installs['events_%wifi'] = installs['ref_hash'].map(eventos.set_index('ref_hash')['wifi'])
installs['event_apps'] = installs['ref_hash'].map(eventos.set_index('ref_hash')['application_id'])
installs['distinct_events'] = installs['ref_hash'].map(eventos.set_index('ref_hash')['kind'])
installs['total_events'] = installs['ref_hash'].map(eventos.set_index('ref_hash')['event_id'])

In [ ]:
del installs['attributed']
del installs['implicit']
del installs['application_id']
del installs['device_model']
del installs['ip_address']
del installs['device_language']

In [ ]:
installs.fillna(0, inplace=True)

In [ ]:
installs = installs.sort_values(by = ['ref_hash', 'created'])

In [ ]:
installs['most_freq_lang'] = installs['most_freq_lang'].astype('float64')

In [ ]:
installs1 = installs[(installs['created'].dt.day < 21)]
installs2 = installs[(installs['created'].dt.day > 18) & (installs['created'].dt.day < 22)]
installs3 = installs[(installs['created'].dt.day > 19) & (installs['created'].dt.day < 23)]
installs4 = installs[(installs['created'].dt.day > 20) & (installs['created'].dt.day < 24)]
installs5 = installs[(installs['created'].dt.day > 21) & (installs['created'].dt.day < 25)]
installsCheck1 = installs[(installs['created'].dt.day > 20) & (installs['created'].dt.day < 23)]
installsCheck2 = installs[(installs['created'].dt.day > 21) & (installs['created'].dt.day < 24)]
installsCheck3 = installs[(installs['created'].dt.day > 22) & (installs['created'].dt.day < 25)]
installsCheck4 = installs[(installs['created'].dt.day > 23) & (installs['created'].dt.day < 26)]
installsCheck5 = installs[(installs['created'].dt.day > 24) & (installs['created'].dt.day < 27)]

In [ ]:
installs1['sc'] = (installs1['created'] - pd.to_datetime('2019-04-18'))/np.timedelta64(1,'s')
installs2['sc'] = (installs2['created'] - pd.to_datetime('2019-04-19'))/np.timedelta64(1,'s')
installs3['sc'] = (installs3['created'] - pd.to_datetime('2019-04-20'))/np.timedelta64(1,'s')
installs4['sc'] = (installs4['created'] - pd.to_datetime('2019-04-21'))/np.timedelta64(1,'s')
installs5['sc'] = (installs5['created'] - pd.to_datetime('2019-04-22'))/np.timedelta64(1,'s')
installsCheck1['sc'] = (installsCheck1['created'] - pd.to_datetime('2019-04-21'))/np.timedelta64(1,'s')
installsCheck2['sc'] = (installsCheck2['created'] - pd.to_datetime('2019-04-22'))/np.timedelta64(1,'s')
installsCheck3['sc'] = (installsCheck3['created'] - pd.to_datetime('2019-04-23'))/np.timedelta64(1,'s')
installsCheck4['sc'] = (installsCheck4['created'] - pd.to_datetime('2019-04-24'))/np.timedelta64(1,'s')
installsCheck5['sc'] = (installsCheck5['created'] - pd.to_datetime('2019-04-25'))/np.timedelta64(1,'s')

In [ ]:
installs1 = installs1.drop_duplicates('ref_hash', keep = 'first')
installs2 = installs2.drop_duplicates('ref_hash', keep = 'first')
installs3 = installs3.drop_duplicates('ref_hash', keep = 'first')
installs4 = installs4.drop_duplicates('ref_hash', keep = 'first')
installs5 = installs5.drop_duplicates('ref_hash', keep = 'first')
installsCheck1 = installsCheck1.drop_duplicates('ref_hash', keep = 'first')
installsCheck2 = installsCheck2.drop_duplicates('ref_hash', keep = 'first')
installsCheck3 = installsCheck3.drop_duplicates('ref_hash', keep = 'first')
installsCheck4 = installsCheck4.drop_duplicates('ref_hash', keep = 'first')
installsCheck5 = installsCheck5.drop_duplicates('ref_hash', keep = 'first')

In [ ]:
del installs1['created']
del installs2['created']
del installs3['created']
del installs4['created']
del installs5['created']
del installsCheck1['created']
del installsCheck2['created']
del installsCheck3['created']
del installsCheck4['created']
del installsCheck5['created']

# Target

En base a los target ids armo los features de esos ids para la predicción.

In [ ]:
target = pd.read_csv('../datos/target_competencia_ids.csv')

In [ ]:
targets_sc = target[target['ref_hash'].str.contains('_sc')]
targets_st = target[target['ref_hash'].str.contains('_st')]

In [ ]:
targets_sc['ref_hash'] = targets_sc['ref_hash'].map(lambda x: str(x)[:-3])
targets_sc['ref_hash'] = targets_sc['ref_hash'].astype(np.int64)

In [ ]:
installs_sin_dup = installs.drop_duplicates('ref_hash')

In [ ]:
targets_sc_with_installs = targets_sc.merge(installs_sin_dup, how = 'left', on = 'ref_hash')

In [ ]:
events.drop_duplicates('ref_hash', inplace= True)

In [ ]:
nulls = targets_sc_with_installs[targets_sc_with_installs['created'].isnull()]
nulls['ref_type'] = nulls['ref_hash'].map(events.set_index('ref_hash')['ref_type'])
nulls['model'] = nulls['ref_hash'].map(events.set_index('ref_hash')['device_model'])
nulls['most_freq_lang'] = nulls['ref_hash'].map(events.set_index('ref_hash')['device_language'])
nulls['most_freq_app'] = nulls['ref_hash'].map(events.set_index('ref_hash')['application_id'])
nulls['ip'] = nulls['ref_hash'].map(events.set_index('ref_hash')['ip_address'])
nulls['events_%wifi'] = nulls['ref_hash'].map(eventos.set_index('ref_hash')['wifi'])
nulls['event_apps'] = nulls['ref_hash'].map(eventos.set_index('ref_hash')['application_id'])
nulls['distinct_events'] = nulls['ref_hash'].map(eventos.set_index('ref_hash')['kind'])
nulls['total_events'] = nulls['ref_hash'].map(eventos.set_index('ref_hash')['event_id'])

In [ ]:
targets_sc_with_installs.update(nulls)
targets_sc_with_installs.head()

In [ ]:
targets_sc_with_installs.fillna({'total_apps' : 0, 'ref_type' : 0, '%attributed' : 0, '%implicit': 0, 'model' : 0,
                                 'most_freq_lang' : 0, 'ip' : 0, 'events_%wifi' : 0, 'event_apps' : 0, 
                                 'distinct_events' : 0, 'total_events' : 0, 'most_freq_app' : 0}, inplace = True)

In [ ]:
del targets_sc_with_installs['created']
del targets_sc_with_installs['obj']

In [ ]:
targets_sc_with_installs['most_freq_lang'] = targets_sc_with_installs['most_freq_lang'].astype('float64')
X = targets_sc_with_installs[['ref_type', 'ref_hash', 'total_apps', '%implicit', '%attributed', 'most_freq_lang', 
                             'most_freq_app', 'model', 'ip', 'events_%wifi', 'event_apps', 'distinct_events', 
                             'total_events']]

Ahora en _X_ tengo los features necesarios para los ids que quiero predecir.

In [ ]:
sc_pred = pd.read_csv('../datos/PrediccionesKNN-RF-XGBoost.txt', header = None)
sc_pred.index = np.arange(1, 8074, 2)
targets_sc['obj'] = sc_pred[0]
target.update(targets_sc)
target

In [ ]:
target.to_csv('../datos/preds-xgboost-rf-knn-nuevos-features.csv', index = False)

# Cargo los modelos entrenados en _PromedioPrediccionesSc.ipynb_
## XBoost

In [ ]:
# Cargo el modelo del archivo
xg_reg = joblib.load('modelos/sc/xg_reg.pkl')  
  
predictions_sc_xgb = xg_reg.predict(X)

## Bagging

In [ ]:
bagging = joblib.load('modelos/sc/bagging.pkl')  
predictions_sc_bagging = bagging.predict(X)

## AdaBoost

In [ ]:
regr = joblib.load('modelos/sc/regr.pkl') 
predictions_sc_adaboost = regr.predict(X)

## LGBM

In [ ]:
gbm = joblib.load('modelos/sc/gbm.pkl')
predictions_sc_lgbm = gbm.predict(X)

## Random Forests

In [ ]:
rf = joblib.load('modelos/sc/rf.pkl')
predictions_sc_rf = rf.predict(X)

## MLP

In [ ]:
nn = joblib.load('modelos/sc/nn.pkl')
predictions_sc_nn = nn.predict(X)

# Promedio las predicciones y actualizo el target

In [ ]:
predictions_sc = np.add(np.add(np.add(np.add(np.add(predictions_sc_xgb,predictions_sc_bagging), predictions_sc_adaboost), predictions_sc_lgbm),predictions_sc_rf), predictions_sc_nn)
predictions_sc = predictions_sc / 6
targets_sc['obj'] = predictions_sc
predicciones_target_sc = target[target['ref_hash'].str.contains('_sc')]
predicciones_target_sc['obj'] = predictions_sc
target.update(predicciones_target_sc)
sc_pred = pd.read_csv('../datos/PrediccionesKNN-RF-XGBoost.txt', header = None)
sc_pred.index = np.arange(1, 8074, 2)
targets_sc['obj'] = sc_pred[0]
target.update(targets_sc)

In [ ]:
target.to_csv('../datos/preds-promedios.csv', index = False)

In [ ]:
target.count()

## Blending

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [ ]:
xg_reg = xgb.XGBRegressor(objective ='reg:squarederror', 
                colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, alpha =10, n_estimators = 11)
regr = AdaBoostRegressor(random_state=0, n_estimators=50, learning_rate=0.1)
rf = RandomForestRegressor(n_estimators=10, criterion='mse', max_depth=5, min_samples_split=2, 
                           min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features='auto', 
                           max_leaf_nodes=None, min_impurity_decrease=0.0, min_impurity_split=None, 
                           bootstrap=True, oob_score=False, n_jobs=None, random_state=None, verbose=0, 
                           warm_start=False)
bagging = BaggingRegressor(KNeighborsRegressor(), max_samples=0.5, max_features=0.5)
gbm = lgb.LGBMRegressor(boosting_type= 'gbdt',
                        objective= 'regression',
                        num_leaves= 31,
                        learning_rate= 0.05,
                        feature_fraction= 0.9,
                        bagging_fraction= 0.8,
                        bagging_freq= 5,
                        verbose= 0)

#### Ventana 1

In [ ]:
X_test, y_test = installsCheck1.iloc[:,:-1], installsCheck1.iloc[:,-1]

In [ ]:
X, y = installs1.iloc[:,:-1], installs1.iloc[:,-1]
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3)
xg_reg.fit(X_train, y_train)
rf.fit(X_train, y_train)
bagging.fit(X_train, y_train)
gbm.fit(X_train, y_train, eval_set=[(X_val, y_val)], eval_metric='l1', early_stopping_rounds=15)

In [ ]:
preds_xgb = xg_reg.predict(X_val)
preds_rf = rf.predict(X_val)
preds_bag = bagging.predict(X_val)
preds_lgbm = gbm.predict(X_val)
test_preds_xgb = xg_reg.predict(X_test)
test_preds_rf = rf.predict(X_test)
test_preds_bag = bagging.predict(X_test)
test_preds_lgbm = gbm.predict(X_test)

In [ ]:
X_val['pred_xgb'] = preds_xgb
X_val['pred_rf'] = preds_rf
X_val['pred_bag'] = preds_bag
X_val['pred_lgbm'] = preds_lgbm
X_test['pred_xgb'] = test_preds_xgb
X_test['pred_rf'] = test_preds_rf
X_test['pred_bag'] = test_preds_bag
X_test['pred_lgbm'] = test_preds_lgbm

In [ ]:
regr.fit(X_val, y_val)
preds = regr.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

#### Ventana 2

In [ ]:
X_test, y_test = installsCheck2.iloc[:,:-1], installsCheck2.iloc[:,-1]
X, y = installs2.iloc[:,:-1], installs2.iloc[:,-1]
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3)
xg_reg.fit(X_train, y_train)
rf.fit(X_train, y_train)
bagging.fit(X_train, y_train)
gbm.fit(X_train, y_train, eval_set=[(X_val, y_val)], eval_metric='l1', early_stopping_rounds=15)

In [ ]:
preds_xgb = xg_reg.predict(X_val)
preds_rf = rf.predict(X_val)
preds_bag = bagging.predict(X_val)
preds_lgbm = gbm.predict(X_val)
test_preds_xgb = xg_reg.predict(X_test)
test_preds_rf = rf.predict(X_test)
test_preds_bag = bagging.predict(X_test)
test_preds_lgbm = gbm.predict(X_test)

In [ ]:
X_val['pred_xgb'] = preds_xgb
X_val['pred_rf'] = preds_rf
X_val['pred_bag'] = preds_bag
X_val['pred_lgbm'] = preds_lgbm
X_test['pred_xgb'] = test_preds_xgb
X_test['pred_rf'] = test_preds_rf
X_test['pred_bag'] = test_preds_bag
X_test['pred_lgbm'] = test_preds_lgbm
regr.fit(X_val, y_val)
preds = regr.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

#### Ventana 3

In [ ]:
X_test, y_test = installsCheck3.iloc[:,:-1], installsCheck3.iloc[:,-1]
X, y = installs3.iloc[:,:-1], installs3.iloc[:,-1]
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3)
xg_reg.fit(X_train, y_train)
rf.fit(X_train, y_train)
bagging.fit(X_train, y_train)
gbm.fit(X_train, y_train, eval_set=[(X_val, y_val)], eval_metric='l1', early_stopping_rounds=15)

In [ ]:
preds_xgb = xg_reg.predict(X_val)
preds_rf = rf.predict(X_val)
preds_bag = bagging.predict(X_val)
preds_lgbm = gbm.predict(X_val)
test_preds_xgb = xg_reg.predict(X_test)
test_preds_rf = rf.predict(X_test)
test_preds_bag = bagging.predict(X_test)
test_preds_lgbm = gbm.predict(X_test)
X_val['pred_xgb'] = preds_xgb
X_val['pred_rf'] = preds_rf
X_val['pred_bag'] = preds_bag
X_val['pred_lgbm'] = preds_lgbm
X_test['pred_xgb'] = test_preds_xgb
X_test['pred_rf'] = test_preds_rf
X_test['pred_bag'] = test_preds_bag
X_test['pred_lgbm'] = test_preds_lgbm
regr.fit(X_val, y_val)
preds = regr.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

#### Ventana 4

In [ ]:
X_test, y_test = installsCheck4.iloc[:,:-1], installsCheck4.iloc[:,-1]
X, y = installs4.iloc[:,:-1], installs4.iloc[:,-1]
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3)
xg_reg.fit(X_train, y_train)
rf.fit(X_train, y_train)
bagging.fit(X_train, y_train)
gbm.fit(X_train, y_train, eval_set=[(X_val, y_val)], eval_metric='l1', early_stopping_rounds=15)

In [ ]:
preds_xgb = xg_reg.predict(X_val)
preds_rf = rf.predict(X_val)
preds_bag = bagging.predict(X_val)
preds_lgbm = gbm.predict(X_val)
test_preds_xgb = xg_reg.predict(X_test)
test_preds_rf = rf.predict(X_test)
test_preds_bag = bagging.predict(X_test)
test_preds_lgbm = gbm.predict(X_test)
X_val['pred_xgb'] = preds_xgb
X_val['pred_rf'] = preds_rf
X_val['pred_bag'] = preds_bag
X_val['pred_lgbm'] = preds_lgbm
X_test['pred_xgb'] = test_preds_xgb
X_test['pred_rf'] = test_preds_rf
X_test['pred_bag'] = test_preds_bag
X_test['pred_lgbm'] = test_preds_lgbm
regr.fit(X_val, y_val)
preds = regr.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

#### Ventana 5

In [ ]:
X_test, y_test = installsCheck5.iloc[:,:-1], installsCheck5.iloc[:,-1]
X, y = installs5.iloc[:,:-1], installs5.iloc[:,-1]
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3)
xg_reg.fit(X_train, y_train)
rf.fit(X_train, y_train)
bagging.fit(X_train, y_train)
gbm.fit(X_train, y_train, eval_set=[(X_val, y_val)], eval_metric='l1', early_stopping_rounds=15)

In [ ]:
preds_xgb = xg_reg.predict(X_val)
preds_rf = rf.predict(X_val)
preds_bag = bagging.predict(X_val)
preds_lgbm = gbm.predict(X_val)
test_preds_xgb = xg_reg.predict(X_test)
test_preds_rf = rf.predict(X_test)
test_preds_bag = bagging.predict(X_test)
test_preds_lgbm = gbm.predict(X_test)
X_val['pred_xgb'] = preds_xgb
X_val['pred_rf'] = preds_rf
X_val['pred_bag'] = preds_bag
X_val['pred_lgbm'] = preds_lgbm
X_test['pred_xgb'] = test_preds_xgb
X_test['pred_rf'] = test_preds_rf
X_test['pred_bag'] = test_preds_bag
X_test['pred_lgbm'] = test_preds_lgbm
regr.fit(X_val, y_val)
preds = regr.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

In [ ]:
test = targets_sc_with_installs[['ref_type', 'ref_hash', 'total_apps', '%implicit', '%attributed', 'most_freq_lang', 'most_freq_app', 'model', 'ip', 'events_%wifi', 'event_apps', 'distinct_events', 'total_events']]

In [ ]:
preds_xgb = xg_reg.predict(test)
preds_rf = rf.predict(test)
preds_bag = bagging.predict(test)
preds_lgbm = gbm.predict(test)

In [ ]:
test['preds_xgb'] = preds_xgb
test['preds_rf'] = preds_rf
test['preds_bag'] = preds_bag
test['preds_lgbm'] = preds_lgbm

In [ ]:
predictions_sc = regr.predict(test)
predictions_sc

In [ ]:
targets_sc['obj'] = predictions_sc
predicciones_target_sc = target[target['ref_hash'].str.contains('_sc')]
predicciones_target_sc['obj'] = predictions_sc
target.update(predicciones_target_sc)
sc_pred = pd.read_csv('../datos/PrediccionesKNN-RF-XGBoost.txt', header = None)
sc_pred.index = np.arange(1, 8074, 2)
targets_sc['obj'] = sc_pred[0]
target.update(targets_sc)

In [ ]:
target.to_csv('../datos/preds-blending.csv', index = False)